In [61]:
import pandas as pd
import numpy as np
import seaborn as sns

In [107]:
def process_company(company_data):
    try:
        m = company_data.money_raised_usd.values
        p = company_data.post_money_valuation_usd.values
        e = company_data.exit_value.values[0]

        if company_data.exit_type.values[0] == 3:
            ret = np.zeros_like(company_data.uuid) - 1
        elif company_data.shape[0] == 1:
            ret = [(m[0] / p[0] * e - m[0]) / m[0]]
        else:
            x0 = m / p # stake of investors at their first round
            x = (p[1:] - m[1:]) / p[1:] # dilution factors
            d = np.cumprod(x)[::-1]
            d = np.insert(d, len(d), 1)
            ret = (e * x0 * d - m) / m
        return pd.DataFrame(data=ret, index=company_data.uuid)
    except:
        return pd.DataFrame(data=[np.nan] * company_data.uuid.size, index=company_data.uuid)


def filter_dates(funding_round):
    return funding_round['announced_on'] < funding_round['exit_date']

In [108]:
process_company(rounds[rounds.org_uuid == "0059ddde-8c8e-2058-f0e4-cf9225ab8ae6"])

,0
uuid,
bba1aab5-4ebb-b321-17a7-7e584ef46ab7,0.377495
c978a78c-f802-0989-6282-4c38c2beabb3,0.010236
886333df-7b04-826b-0c13-70e9e3d575ba,0.719773
44aea3c6-52e0-bc3e-0666-31ece75c497b,2.296062


In [110]:
res = rounds.groupby('org_uuid').apply(process_company)

In [80]:
rounds_data = pd.read_csv('obs_predicted_data.csv')

In [81]:
orgs = pd.read_csv("../../data/crunchbase/organizations.csv")
acqs = pd.read_csv("../../data/crunchbase/acquisitions.csv")
ipos = pd.read_csv("../../data/crunchbase/ipos.csv")

In [82]:
ipos = ipos[['org_uuid', 'went_public_on', 'valuation_price_usd']]
acqs = acqs[['acquiree_uuid', 'acquired_on', 'price_usd']]

In [83]:
rounds = rounds_data[['uuid', 'org_uuid', 'announced_on', 'money_raised_usd', 'post_money_valuation_usd']].copy()

ipos.columns = ['org_uuid', 'exit_date', 'exit_value']
acqs.columns = ['org_uuid', 'exit_date', 'exit_value']

clos = pd.DataFrame(orgs[['uuid', 'closed_on']].dropna())
clos.columns = ['org_uuid', 'exit_date']

ipos["exit_type"] = 1
acqs["exit_type"] = 2
clos["exit_type"] = 3
clos["exit_value"] = np.nan

exits = pd.concat([acqs, ipos, clos]).set_index("org_uuid")

In [84]:
rounds["exit_date"] = pd.to_datetime(rounds.org_uuid.map(exits["exit_date"].to_dict()))
rounds["exit_type"] = rounds.org_uuid.map(exits["exit_type"].to_dict()).fillna(value=4).astype(int)
rounds["exit_value"] = rounds.org_uuid.map(exits["exit_value"].to_dict())

In [85]:
ref_date = '1926-01-01'
rounds.announced_on = pd.Timestamp(ref_date) + rounds.announced_on * pd.Timedelta('1s')

In [86]:
private = rounds[rounds.exit_type == 4]
rounds = rounds[(rounds.announced_on < rounds.exit_date)]
rounds = rounds.sort_values(by=['org_uuid', 'announced_on'])

In [118]:
rounds['return_usd'] = rounds.uuid.map(rounds.groupby('org_uuid').apply(process_company).droplevel(0)[0]) + 1

In [119]:
rounds = rounds.drop(columns=['uuid'])
rounds.insert(1, 'group_num', np.nan)
rounds.insert(2, 'seg_num', np.nan)

rounds.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
rounds = rounds[rounds.round_date > '1990-01-01']

In [120]:
private = private.drop(columns=['uuid'])
private.insert(1, 'group_num', np.nan)
private.insert(2, 'seg_num', np.nan)
private.insert(9, 'return_usd', np.nan)

private.columns = ['company_num', 'group_num', 'seg_num', 'round_date', 'raised_usd',
                  'postvalue_usd', 'exit_date', 'exit_type', 'exit_value', 'return_usd']
private = private[private.round_date > '1990-01-01']
private.exit_date = np.nan

In [121]:
complete = pd.concat([rounds, private])
complete.exit_date = pd.to_datetime(complete.exit_date)
complete[complete.return_usd == 0] = np.nan
complete.to_csv('../../data/data.csv', index=False)

In [122]:
complete

,company_num,group_num,seg_num,round_date,raised_usd,postvalue_usd,exit_date,exit_type,exit_value,return_usd
8839,000cff2d-58df-0a9a-97ea-6b73ed9ec601,NaN,NaN,2016-07-21,100000.0,1.120240e+06,2020-03-06,2.0,NaN,NaN
8850,0012f446-a31b-9eb8-4fa0-448ed0414112,NaN,NaN,2000-03-07,14000000.0,7.059858e+07,2005-06-03,2.0,NaN,NaN
8874,00251aad-6e34-0898-8a72-fe8864a90f92,NaN,NaN,2012-10-04,650532.0,3.238880e+06,2016-11-08,2.0,NaN,NaN
8881,002aac96-79d4-a8f3-8ef0-d44fb240951d,NaN,NaN,2000-05-12,3000000.0,1.126037e+07,2006-11-09,2.0,NaN,NaN
8886,002e0b73-ab4e-bfd5-0d9a-e6df8fede446,NaN,NaN,2016-01-11,45000000.0,2.104294e+08,2020-12-15,1.0,577580000.0,2.442438
...,...,...,...,...,...,...,...,...,...,...
120143,fffe3b0b-8545-4414-9c0e-a3f961c4e420,NaN,NaN,2017-02-07,14523274.0,8.525829e+07,NaT,4.0,NaN,NaN
120144,ffff2bea-7e44-4563-b311-65daadb683cd,NaN,NaN,2010-12-07,451407.0,2.387976e+06,NaT,4.0,NaN,NaN
120145,ffff2bea-7e44-4563-b311-65daadb683cd,NaN,NaN,2013-04-25,1616916.0,8.092737e+06,NaT,4.0,NaN,NaN
120146,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,NaN,NaN,2016-05-15,210000.0,2.649445e+06,NaT,4.0,NaN,NaN
